In [1]:
!pip install transformers datasets tensorflow scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00


In [4]:
import json
import pandas as pd

# Load JSON dataset
with open('/content/drive/MyDrive/Geeks.exe/training_dataset.json', 'r') as f:
    data = json.load(f)

# Convert JSON to DataFrame
df = pd.DataFrame(data["transactions"])

# Display dataset structure
print(df.head())
print("Columns in dataset:", df.columns)


  transaction_id   merchant_name  amount        category   subcategory
0           T001  Domino's Pizza   500.0   Food & Dining   Restaurants
1           T002      Big Bazaar  1500.0   Food & Dining     Groceries
2           T003            Uber   300.0  Transportation  Ride-sharing
3           T004      Indian Oil  2000.0  Transportation          Fuel
4           T005    Reliance Jio  1000.0         Housing     Utilities
Columns in dataset: Index(['transaction_id', 'merchant_name', 'amount', 'category', 'subcategory'], dtype='object')


In [5]:
from sklearn.preprocessing import LabelEncoder

# Create text input and labels
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces
df["text"] = "Transaction at " + df["merchant_name"]
df = df[["text", "category"]]

# Encode labels
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["category"])

# Get number of unique categories
num_labels = len(label_encoder.classes_)

# Show category mapping
print(dict(zip(label_encoder.classes_, range(num_labels))))


{'Education': 0, 'Entertainment': 1, 'Food & Dining': 2, 'Health & Fitness': 3, 'Housing': 4, 'Miscellaneous': 5, 'Paid to person in contact': 6, 'Personal Expenses': 7, 'Savings & Investments': 8, 'Shopping': 9, 'Shopping & Groceries': 10, 'Transportation': 11, 'Travel': 12}


<ipython-input-5-85030c9975d9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = label_encoder.fit_transform(df["category"])


In [6]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize dataset
def tokenize_data(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=64, return_tensors="tf")

# Convert text column to tokenized format
tokenized_inputs = tokenize_data(df["text"].tolist())
print(tokenized_inputs
)
# Extract input tensors
input_ids = tokenized_inputs["input_ids"]
attention_masks = tokenized_inputs["attention_mask"]
labels = df["label"].values


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': <tf.Tensor: shape=(238, 64), dtype=int32, numpy=
array([[  101, 12598,  2012, ...,     0,     0,     0],
       [  101, 12598,  2012, ...,     0,     0,     0],
       [  101, 12598,  2012, ...,     0,     0,     0],
       ...,
       [  101, 12598,  2012, ...,     0,     0,     0],
       [  101, 12598,  2012, ...,     0,     0,     0],
       [  101, 12598,  2012, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(238, 64), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(238, 64), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [7]:
import tensorflow as tf

# Convert to TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices(({"input_ids": input_ids, "attention_mask": attention_masks}, labels))

# Split dataset (80% Train, 20% Test)
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)

# Shuffle and batch dataset
batch_size = 16
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [8]:
from transformers import TFBertForSequenceClassification

# Load BERT model with classification head
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_labels):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_labels, activation='softmax')

    def call(self, inputs):
        outputs = self.bert(inputs)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits

In [17]:
classifier = BERTForClassification(model.bert, num_labels)

classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

In [20]:
history = classifier.fit(
    train_dataset,epochs=15,    validation_data=test_dataset,

)

Epoch 1/15
12/12 [==============================] - 3s 249ms/step - loss: 0.5393 - accuracy: 0.9105 - val_loss: 0.7679 - val_accuracy: 0.7917
Epoch 2/15
12/12 [==============================] - 3s 288ms/step - loss: 0.4523 - accuracy: 0.9579 - val_loss: 0.7325 - val_accuracy: 0.7917
Epoch 3/15
12/12 [==============================] - 3s 251ms/step - loss: 0.3917 - accuracy: 0.9579 - val_loss: 0.6932 - val_accuracy: 0.7708
Epoch 4/15
12/12 [==============================] - 3s 232ms/step - loss: 0.3534 - accuracy: 0.9684 - val_loss: 0.6851 - val_accuracy: 0.7708
Epoch 5/15
12/12 [==============================] - 3s 232ms/step - loss: 0.2827 - accuracy: 0.9842 - val_loss: 0.6588 - val_accuracy: 0.7917
Epoch 6/15
12/12 [==============================] - 3s 228ms/step - loss: 0.2717 - accuracy: 0.9737 - val_loss: 0.7177 - val_accuracy: 0.7708
Epoch 7/15
12/12 [==============================] - 3s 233ms/step - loss: 0.2358 - accuracy: 0.9737 - val_loss: 0.6464 - val_accuracy: 0.8125
Epoch 

In [23]:
result = classifier.evaluate(test_dataset)
print(f"Test Accuracy: {result[1] * 100:.2f}%")


3/3 [==============================] - 0s 118ms/step - loss: 0.6690 - accuracy: 0.7708
Test Accuracy: 77.08%


In [32]:
# Sample test transaction
test_text = ["Transaction at 2039203929 "]

# Tokenize the input
test_tokens = tokenizer(test_text, padding="max_length", truncation=True, max_length=64, return_tensors="tf")

# Extract input tensors
test_input = {"input_ids": test_tokens["input_ids"], "attention_mask": test_tokens["attention_mask"]}


In [33]:
import numpy as np

# Make prediction
predictions = classifier.predict(test_input)

# Get the category with the highest probability
predicted_label = np.argmax(predictions, axis=1)[0]

# Decode the predicted label back to category name
predicted_category = label_encoder.inverse_transform([predicted_label])[0]

print("Predicted Category:", predicted_category)


1/1 [==============================] - 0s 47ms/step
Predicted Category: Miscellaneous
